# Training - AI safety for b-tagging@CMS edition
Now we are at the heart of the project - training the models with lots of customizable options. For this part, the relevant code is split among several files and you will also notice the first batch script in this directory.

<div align="center">
<img src="https://www.meme-arsenal.com/memes/4db8b12624416519c13c5c929e89b09b.jpg" width="400"/><br>
    Fig. 1 <a href="https://www.meme-arsenal.com/en/create/meme/4336524">[from meme-arsenal.com]</a>
</div>

The tutorial is split into four parts:
- [Prerequisites](#prerequisites)
- [How the training code works](#code)
- [Perform training yourself](#perform)
    - [On interactive node](#interactive)
    - [On batch system](#batch)
- [Some tasks](#tasks)

## Prerequisites<a name="prerequisites"></a>
This write-up assumes that you have done all cleaning and preprocessing steps to produce the training data. Further, make sure you are able to submit jobs to the batch system, preferably under a dedicated project. The introductory slides have instructions for this prcedure and how to get started.

Relevant parts of the thesis for the training in particular: 2.1 and 2.4

## How the training code works<a name="code"></a>
First, have a look at the files in this directory.

The `fast_tensor_data_loader.py` is meant to facilitate the dataloading inside the training script and is based on an original version that is part of pytorch-tabular, and has been adapted a bit to combine a fast runtime with rather efficient memory usage. You don't need to understand this code in detail, just remember it's there to make the manual batching a bit easier / faster, while also being able to shuffle jets. Not using this (or an equivalent implementation) would increase the time per epoch from like 11 minutes to two days, I tested this a while back...

Next, the `focal_loss.py` script. Again, I adapt some code that I found on the web, but in this case, to create a custom loss function, which is able to perform a multi-class reweighting according to how easy one can classify the samples. Read more about this in the thesis, that's section 2.3.3. Actually, only the class is used, which is derived from pytorch's `nn.Module`, the 'standalone' `focal_loss` function is not used currently, but would do something similar. In fact, to understand how loss functions work, maybe it would be a good idea to start reading about the standard (categorical) cross entropy loss first, it's documented by pytorch and practically every resource you can find mentions it. Think of the focal loss as a more advanced cross entropy, even more advanced than just a class-reweighted cross entropy.

Inside `jet_reweighting.py` you will find a class that could be used for a reweighted training based on resampling the jets, not reweighting a sample inside the loss function (loss weighting). It makes more sense if you read about the reweighting in sections 2.3.1 and 2.3.2, and it is currently not the preferred option (I didn't even bother to implement the two weighting techniques (average versus flat) because the resampling is deprecated). Think of this as a time- and memory-expensive way to reweight the jets by drawing them from a multinomial distribution, not very efficient. Just forget about it if it looks a bit inconvenient to you. Update the paths if you'd like to use it nevertheless.

For the training itself (now speaking about the *actual* scripts to keep in mind), there is three files that you shuold understand at least superficially to run the training, and more in-depth if you want to adjust them later on. So there is `submit_training.py`, `training.sh` and `training.py`. What you would normally do in a standard setting is to use the first script out of these to submit jobs to the batch system, each job runs it's own `.sh` script and from those, the actual training will be started (part of `training.py`). The training script could also be used as standalone, and in fact this is a good way to check if your code would *in principle* work also inside a job. But more on the practical side will come later. Of course you can now quickly skim the three remaining scripts to get an overview, but I'd like to explain the `training.py` in a bit more detail now.

I guess by now you are familiar with the general setup such that referring to the lines in the code is enough for you to understand my explanations, copying every bit into this tutorial would be a bit redundant (and too long).

Lines 1-36: There are some packages that you already know from previous script, but there is also something new that has been introduced for the training in particular. There is a special function in sklearn with which class weights can be calculated. While this is valid option, we don't need it anymore because we calculated the weights on our own, it's leftover from an earlier version of the script. We need some more functionality of pytorch, for example `.nn` which concerns the model with its layers and basic methods related to the training. `torch.utils.data` has some additional helpful classes for the dataloading and sampling, but not all will be used in the current version of the script. `time` will be helpful for keeping track of the training time, `random` could be used to randomly select files, but again, it's not used currently (instead there are the samplers which do this task). New packages that were not explained before: `os` and `sys` (mainly to automatically create new folders per training setup, such that the stored models do not overwrite the previous models); then there are our custom modules concerning reweighting, dataloading and loss function, as well as the implementation of the FGSM attack used when doing adversarial training. You should already adjust the path to your own path if possible, but understanding this can come later when we explain the attacks in more detail.

The code in line 41-46 is meant to make the script ready for gpu usage, if there is a gpu available. Otherwise the model and tensors shall stay on the cpu.

The first timestamp is set in line 52.

Lines 56-134: parser with custom options for the training setup, and creation of some str to be added to the output paths later to be able to differentiate between the different setups and have them all independent of each other, also regarding the weighting method, focal loss with gamma and potentially also alpha, adversarial training or not.

Lines 137-214: setup random seeds for the initialization of the model for the first epoch (identically for all trainings for comparability of setups), create new directories on `/home` (backup) and `/hpcwork` (fast) - adjust all paths there to your own ones! FOr this, make sure to first create the directories `train_models/saved_models` on both partitiions; set batchsize and initial learning rate, print setup to command line and write to custom logfile (create directory that will hold the logs first and update path then!); in case of a minimal setup (when first starting out with the training to see if there are errors, before submitting to SLURM, see later), load only some of the file paths, otherwise, load them all, also load only the correct sample weights according to the weighting method. You see, all those need you own paths, like everytime. :)

Lines 216-243: loading training samples, together with training targets, based on the weighting method and type of the sampler, in some cases, everything needs to be part of the same big data set, if no weighting will be applied, it doesn't matter and the inputs and targets can be loaded individually.

Next up there is some deprecated code in lines 246-262 when loading the validation samples happened prior to the first training epoch, ignore this.

Lines 264-268: load the bins for training samples, but again, only necessary for a deprecated weighting method.

Another timestamp in line 271.

Lines 272-306: outdated data loading, ignore.

Lines 307-327: depending on the weighting and sampling method, create corresponding dataloaders (that's a special way of presenting the data to the model inside the training loop, which comes later)

Line 329-338: not necessary here anymore, because validation is loaded only after first epoch.

Lines 340f.: get the number of training samples.

Lines 344-347: not used (validation later).

Lines 350f.: if you only want to test if the dataloading works, stop the script at this stage (you could delete the `#` in front to stop the execution there, but normally you want to train the model during training :-))

Lines 353-368: that's the model with all layers, activation functions and dropout.

Lines 371-373: in case that's not the first epoch, the script will load the previous epoch as specified with the arguments, and adjust the state of the parameters. Adjust the paths.

Lines 375-385: print the model, then there is some outdated tests with which I hoped to distribute the model on several parallel processes, but that turned out to be too difficult to work properly, or it was even slower without correct setup of MPI etc.; the model is shifted to the device (either gpu or cpu, as explained above).

Lines 387-422: first, there are some old ways to create the loss function, but the current way start in line 409. There is a check for the weighting method, because this will have an impact on which loss function is necessary. Similar for the focal loss, it will be either used, or not (then: cross entropy). The reduction parameter is set to `None` because the reduction will only come later after the loss weighting in case of the first two weighting methods, it means that the return value is still a vector for all samples, and only after the weighting the type changes to a scalar. Without reweighting, the reduction is already part of the calculation of the cross entropy loss. The optimizer is the same for all cases, and it uses a given learning rate.

Lines 425-436: regarding the the optimizer, in case of the first epoch, this prints the initial learning rate, otherwise, the optimizer is set to the state of the previous checkpoint. Originally, I had the decaying rate there, but see the updated version below.

Lines 438-444: this implements a decaying learning rate and is a function that will get called during every epoch of the training to update the optimizer. Look at section 2.1.2 of the thesis for an explanation.

Lines 448-459: some more timestamp, and the initialization of the lists that save the loss for training and validation, the implementation of early stopping with (non-optimized) parameters, a list that will hold the timestamps per epoch, and another line is written to the logfile. Path needs to be updated.

Lines 464-624: the main training loop that runs over all epochs of the training. Again, some timestamp is added. Then, there is a check that concerns how many epochs have been done already (taking previous jobs into account as specified by `prev_epochs`, but also including the 'new' epochs of the currently running script. If it's not the first epoch, update the learning rate. The initial `running_loss` is set to zero and will be updated during the training with all batches taken into account. The model gets told that there will be the training coming up soon, this adjusts the internal setup (e.g. concerning dropout layers, they are active during training, but not during validation). In line 470, one extracts batches from the trainloader (`b` counts the batches, and the tuple (`i`,`j`) yields inputs and targets, respectively). In case it's the second batch of the first epoch, there is some more printout with information how long the first batch took. More is written to the logfile (adjust path). Next, there is again a check regarding the weighting method. This is because in the loss weighting case, one needs to extract the sample weights before putting `i` into to model (this holds the inputs as well as the assigned weights, so one more column that what goes into the model). Everything is transmitted to the device (e.g. gpu). The `zero_grad()` method of the optimizer is called to reset the gradients. Then some old code you can ignore. And depending on the chosen `epsilon`, there will be adversarial training, or not (the trick is to give a negative epsilon as argument to tell the script that you want to run the basic training on raw inputs). In case of the basic training, the model is evaluated on the inputs `i` (but they are transformed to float, because that's the type with which the model can deal with). In case of adversarial training, first, the attack will be applied on the inputs `i`, only then the model is evaluated. More on the attack later. With the help of the criterion (loss function) one measures how 'bad' the predictions (`output`) are compared to the true targets `j`. This is stored inside the variable `loss`, but note that this is (in case of loss weighting) a non-reduced value. When this has been done, one can delete the inputs and targets from memory. In case you'd want to recreate the focal loss yourself without the additional module, there is some code that does this. But ignore the code inside the block comment for now. Instead, we already have that implemented in the criterion itself. In lines 517ff., according to the weighting methods that use loss weighting, the loss (vector) is multiplied with normalized sample weights. If there is no loss weighting, this is not necessary. In any case, now (finally!) the backpropagation happens by calling `.backward()` on the (scalar) loss. With those gradients in mind, the optimizer is updated, or rather, the parameters of the model, according to what the optimizer calculates based on the gradients of the loss funciton. In lilne 524, the loss (interpreted as a single, scalar value) is decoupled from its gradients by calling `.item()` - this frees again some memory. The current (batch)-loss is added to the `running_loss`introduced above. To free even more memory, also the predictions are now deleted. This goes on with all batches, until the trainloader is `empty`. This marks the end of the first training epoch, after which the `else` case takes over. There, we don't need the training loss any more, so it can be deleted. If the validation happens for the first time (`e==0`), we need to first of all set a timestamp to compare the times later, then some logs are written (check path!), and it's very important to load the validation samples now. Everything (except for the weights which are not necessary for validation) is similar to the loading of the training samples. Even the creation of the dataloader is just adapted, changing `train` to `val` and should now be self-explanatory, after looking at the training in detail. Note something that is now different, compared to the training: no matter if it's the first or any later epoch, there is some context defined where torch does not enforce the creation of gradients (`with torch.no_grad()`). This is because for validation, we are not interested in gradients, only the loss value itself from the evaluation counts. Even though we are not reweighting for validation, if the loss has been created for the reweighting-version, we still need to recreate a scalar loss, also for validation. The following checks concern the early stopping, should the validation loss not improve for longer than the specified number of epochs, the training loop will be interrupted there. In practice, this will not be used currently. There is some printout which shows the training and validation loss as function of epoch. Note how lines 622f. save the model (the full checkpoint concerning the epoch, the model parameters, the state of the optimizer, the training and validation loss) to disk, first to the `hpcwork` partition and then also to `home` (your paths!). When the training loop is done, there will be some timestamp again and we print some more information on the training. Finally, another copy of the model is saved (update path!), but there, only the model itself (which can be used for inference, but not really to train it any further). This saved checkpoint comes at a much smaller file size, but also with less information. Finally, after the last timestamp, there is a printout of the time it took for every epoch of the training or the whole script.

This was the explanation of the main training code which runs in any case, no matter if you run the training interactively or in a batch job.

## Perform training yourself <a name="perform"></a>
With the theoretical knowledge of the training you can now start training a basic model (with a limited training set or using all available samples in a second step, then with a job).
### On interactive node <a name="interactive"></a>
To start the training on an interactive node, there are two options: doing so with non-gpu nodes or on gpu nodes. There are for example the "normal" nodes: login18-1.hpc.itc.rwth-aachen.de (or -2,-3,-4) or the gpu ones: login18-g-1.hpc.itc.rwth-aachen.de (or -2). They are just meant for rather quick checks of your code, meaning that there is a timelimit. Additionally, using the gpu (one of two available gpus per node, I think) will block that device for other users in the current setup, so that should also only be done for a short time. If you go over the timelimit you probably receive some e-mail and you will be warned about this, but don't worry, nothing bad will happen if you don't do this repeatedly. :) The timelimit is sometimes a bit strange, it's not always 20 minutes or 10 minutes for some reason. In any case, no matter which node you choose, activate the conda environment, and load necessary modules if you want to use the gpu:
```
module unload intelmpi; module switch intel gcc
module load cuda/11.0
module load cudnn
```
Then, start the training with a simplified setup, taking only a limited number of files into account:
```
python training.py 20 0 1 _ptetaflavloss 0.001 -1 yes yes yes 25 equal equal equal equal -1
```

In [ ]:
# hint: you can also start the code from here, inside the notebook (use the ! in front),
# that way you can keep track of what you did interactively and the stdout will be part of your notebook to check later (and maybe for copy-pasting commands ;-))
!python training.py 20 0 1 _ptetaflavloss 0.001 -1 yes yes yes 25 equal equal equal equal -1

### On batch system <a name="batch"></a>
If there were no errors while running a reduced version of the training on the interactive node, you can now start submitting jobs to the batch system! There are a couple of options, using the `sbatch` command directly (from the command line), or a bit more convenient, by using `submit_training.py`. The second option is preferred and will be described.

The structure looks similar to the normal `training.py` script, considering the arguments that go into the parser. But note the difference: here, you do not *have* to specify every argument, if the defaults are ok for your use case. Additionally, there is the subprocess module which handles the submission of batch jobs from inside the python script (so you don't call `sbatch` yourself, it's done by the subprocess). You need to create another directory that will collect all the output logs by slurm, so update the `logPath` to your one (in line 53). Also, update line 57 such that you can point to the place where the training scripts are located. Lines 60-81 are used to 'calculate' some of the parameters for the job like time and memory, based on what number of datasets you specified or for how many epochs the training should run. It's not perfectly optimized, but should be good enough to ensure that you do not waste computing resources, while still maintaining a minimum of the requirements to don't produce failed jobs. In line 84-88, the submit command will be built: it uses the calculated parameters via string-formating and generates an unambiguos name to identify the jobs later. With the help of the `--export` flag, the arguments are transmitted to the actual `.sh`-script that runs on the worker node. This command is written to the command line for you to check if everything is ok. If you see no mistakes, type `y` and the job will be submitted. Should there be a problem with the submission, you will be informed. In case you see that you had a typo or something else was wrong, just don't type `y` and start the submission with a new try. So that's what should happen if you use the command specified below. But don't do it yet, first you want to customize the `.sh`-script.

Inside `training.sh` you see the basic setup for SLURM. It starts with the `#` commands. The ones that are now controlled by the submit script (not from `.sh` anymore) are commented out, meaning, there is a space between the `#` and the command (e.g. line 7). When `#SBATCH` doesn't have this space, it will be used. Leave the `#` lines as they are, except for the ones that concern the project (if yours is different) and the mail (this will definitly be different, and maybe you don't even want to receive mails, in that case, modify the mail-type or just add spaces for those `#` lines after the `#`-sign. Note that currently, the script is configured to use a gpu (line 21). After all those `#` lines, there are the ones that might be more familiar to you. First, adjust the path under which the HPC is going to search for the training script (your path). Lines 28-32 can stay, but if your conda environment has a different name (very likely), also modify line 32. Line 33 can stay as it is.

Now to submit, you can do
```
python submit_training.py -f 20 -p 0 -a 1 -w '_ptetaflavloss' -d 0.001 -j -1 -m yes -l yes -fl yes -g 25 -al1 equal -al2 equal -al3 equal -al4 equal -eps -1
```




## Some tasks <a name="tasks"></a>
Did the minimal-working-example *work*? Check if you can access the logfiles, check if the model has been saved (in both partitions) and check your mails, in case you wanted to receive them.
- Train the model for more than one epoch, you will have to specify `-p 1` and `-a 99` if you, for example, want to add 99 epochs to the training after one epoch has been done previously. In case you are fine with the defaults, you can also run the submit script without specifying every parameter (e.g. the four alphas can get annoying, you probably don't want to use them anyway because of our own reweighting).
- Train the model on the full data set, e.g. use `-f 278`. There, start again slowly with only one epoch (sometimes the interactive timelimit is enough for that first epoch to see if everything works), and then use several iterations, each consisting of around 30 epochs, they will already take some time (a couple of hours on all available training jets). I stopped at epoch 200.

With the first trained model, you could already try out some evaluation with the scripts of the `evaluate` directory as a next step. Some ideas that concern modifying or improving the training directly (and which not important now, when starting out):
- Use different hyperparameters for those that are already implemented with the parser, for example, change the $\gamma$ a bit, or do adversarial training (although you'll understand the adversarial training better when looking at the `attack` directory).
- For later: extend the arguments such that you can also vary more parameters of the training setup, like the batchsize. Note: if you do this, make sure to include this information somewhere in the paths of the saved models, because otherwise, you'd overwrite the previous models, which would be sad. This doesn't have priority now, though, evaluating the performance will be much more important now at the beginning of your studies, optimizations can come later.
- Much later: if you also think that the parameters have much too long names and that the paths become too long, try figuring out some way to identify the setup unambiguosly without the super long arguments, e.g. using shorter abbreviations, or storing models with concecutive numbers plus auxiliary files that only store the setup (e.g. as json), or some code / hash... just some idea to make our lives easier in the future.